In [32]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [33]:
# Sort the data alphabetically for use later
school_data_complete.sort_values(by=['school_name'], ascending = True, inplace = True)
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
19584,19584,Tammie Fox,F,11th,Bailey High School,82,92,7,District,4976,3124928
21193,21193,Jennifer Murray,F,9th,Bailey High School,88,89,7,District,4976,3124928
21192,21192,Lisa Pineda,F,9th,Bailey High School,86,67,7,District,4976,3124928
21191,21191,Cameron Miller,M,11th,Bailey High School,70,75,7,District,4976,3124928
21190,21190,Thomas Rasmussen,M,12th,Bailey High School,77,82,7,District,4976,3124928


## District Summary

In [10]:
# District Summary

# Create variables to hold all series to be merged later into a dataframe
total_schools = school_data_complete.school_name.nunique()
total_students = school_data_complete.student_name.count()
total_budget = school_data_complete[['school_name', 'budget']].drop_duplicates(subset = 'school_name')['budget'].squeeze().sum()
average_math_score = school_data_complete[['school_name', 'math_score']]['math_score'].mean()
average_reading_score = school_data_complete[['school_name', 'reading_score']]['reading_score'].mean()
percent_math_pass =  len(school_data_complete[school_data_complete.math_score >= 70]) / school_data_complete.math_score.count() * 100
percent_reading_pass =  len(school_data_complete[school_data_complete.reading_score >= 70]) / school_data_complete.reading_score.count() * 100
overall_pass = len(school_data_complete[(school_data_complete.math_score >= 70) & (school_data_complete.reading_score >= 70)]) / school_data_complete.math_score.count() * 100

# merging all series into a single dataframe
school_data_summary = pd.DataFrame([
  [total_schools, f'{total_students:,}', f'${total_budget:,.2f}', average_math_score, average_reading_score, percent_math_pass, percent_reading_pass, overall_pass],
],
  columns=['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
)

school_data_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [19]:
# School Summary

# Create variables to hold all series to be merged later into a dataframe
school_name = school_data_complete['school_name'].drop_duplicates().reset_index(drop = True)
school_type = school_data_complete[['school_name', 'type']].drop_duplicates(subset = 'school_name')['type'].squeeze().reset_index(drop = True)
total_students = school_data_complete[['school_name', 'size']].drop_duplicates(subset = 'school_name')['size'].squeeze().reset_index(drop = True)
tot_school_budget = school_data_complete[['school_name', 'budget']].drop_duplicates(subset = 'school_name')['budget'].squeeze().reset_index(drop = True)

per_student_budget = (tot_school_budget / total_students).reset_index(drop = True)
ave_math_score = school_data_complete.groupby(['school_name']).math_score.mean().reset_index(drop = True)
ave_reading_score = school_data_complete.groupby(['school_name']).reading_score.mean().reset_index(drop = True)
percent_Passing_Math = school_data_complete.groupby('school_name')['math_score'].apply(lambda x: ((x>=70).sum())/x.count() * 100).reset_index(drop = True)
percent_Passing_Reading = school_data_complete.groupby('school_name')['reading_score'].apply(lambda x: ((x>=70).sum())/x.count() * 100).reset_index(drop = True)

# extract student count values to be used to compute pverall passing percentage
student_count_by_school = school_data_complete.school_name.value_counts()

overall_Passing = school_data_complete.loc[(school_data_complete.math_score>=70) & (school_data_complete.reading_score>=70)]
overall_Passing = (overall_Passing.school_name.value_counts() / student_count_by_school * 100).reset_index(drop = True)

# merging all series into a single dataframe
school_summary = {'School Name': school_name, 'School Type': school_type, 'Total Students': total_students, 'Total School Budget': tot_school_budget.map('${:,.2f}'.format),
                  'Per Student Budget': per_student_budget.map('${:,.2f}'.format), 'Average Math Score': ave_math_score, 'Average Reading Score': ave_reading_score,
                  '% Passing Math': percent_Passing_Math, '% Passing Reading': percent_Passing_Reading, '% Overall Passing': overall_Passing}
school_summary = pd.DataFrame(school_summary).reset_index(drop = True)
school_summary = school_summary.set_index('School Name')
school_summary.index.name = None

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [12]:
# Top Performing Schools (By % Overall Passing)
top_performing_schools = school_summary.sort_values(by = '% Overall Passing', ascending = False).iloc[:5,:]
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [20]:
# Bottom Performing Schools (By % Overall Passing)
bottom_performing_schools = school_summary.sort_values(by = '% Overall Passing', ascending = True).iloc[:5,:]
bottom_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [14]:
# Math Scores by Grade
grade_list = school_data_complete['grade'].unique()
grade_list
ninth_math = school_data_complete.loc[(school_data_complete['grade'] == '9th'), ['school_name', 'math_score', 'grade']].groupby('school_name').math_score.mean()
tenth_math = school_data_complete.loc[(school_data_complete['grade'] == '10th'), ['school_name', 'math_score', 'grade']].groupby('school_name').math_score.mean()
eleventh_math = school_data_complete.loc[(school_data_complete['grade'] == '11th'), ['school_name', 'math_score', 'grade']].groupby('school_name').math_score.mean()
twelfth_math = school_data_complete.loc[(school_data_complete['grade'] == '12th'), ['school_name', 'math_score', 'grade']].groupby('school_name').math_score.mean()

# Merging grade series into dataframe
school_by_math_grades = pd.concat([ninth_math, tenth_math, eleventh_math, twelfth_math], axis=1)
school_by_math_grades.rename(columns={'math_score': '9th', 'math_score': '10th', 'math_score': '11th', 'math_score': '12th'}, inplace= True)
school_by_math_grades.columns = ['9th', '10th', '11th', '12th']
school_by_math_grades.index.name = None

school_by_math_grades

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [15]:
# Reading Score by Grade
ninth_reading = school_data_complete.loc[(school_data_complete['grade'] == '9th'), ['school_name', 'reading_score', 'grade']].groupby('school_name').reading_score.mean()
tenth_reading = school_data_complete.loc[(school_data_complete['grade'] == '10th'), ['school_name', 'reading_score', 'grade']].groupby('school_name').reading_score.mean()
eleventh_reading = school_data_complete.loc[(school_data_complete['grade'] == '11th'), ['school_name', 'reading_score', 'grade']].groupby('school_name').reading_score.mean()
twelfth_reading = school_data_complete.loc[(school_data_complete['grade'] == '12th'), ['school_name', 'reading_score', 'grade']].groupby('school_name').reading_score.mean()

# Merging grade series into dataframe
school_by_reading_grades = pd.concat([ninth_reading, tenth_reading, eleventh_reading, twelfth_reading], axis=1)
school_by_reading_grades.columns = ['9th', '10th', '11th', '12th']
school_by_reading_grades.index.name = None

school_by_reading_grades

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [21]:
# Scores by School Spending
scores_by_school_spending = {'Per Student Budget': per_student_budget, 'Average Math Score': ave_math_score, 'Average Reading Score': ave_reading_score,
                  '% Passing Math': percent_Passing_Math, '% Passing Reading': percent_Passing_Reading, '% Overall Passing': overall_Passing}

# convert to dataframe
scores_by_school_spending = pd.DataFrame(scores_by_school_spending).reset_index(drop = True)

# grab variables for binning purpose
labels = ['<$585', '$585-630', '$630-645', '$645-680']
bins = [0, 585, 630, 645, 680]
scores_by_school_spending['Spending Ranges (Per Student)'] = pd.cut(scores_by_school_spending['Per Student Budget'], bins = bins, labels = labels, include_lowest = True)

del scores_by_school_spending['Per Student Budget']
scores_by_school_spending = scores_by_school_spending.groupby('Spending Ranges (Per Student)').mean().round(2)

scores_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [22]:
# Scores by School Size
scores_by_school_size = {'Total Students': total_students, 'Average Math Score': ave_math_score, 'Average Reading Score': ave_reading_score,
                  '% Passing Math': percent_Passing_Math, '% Passing Reading': percent_Passing_Reading, '% Overall Passing': overall_Passing}

# convert to dataframe
scores_by_school_size = pd.DataFrame(scores_by_school_size).reset_index(drop = True)

# grab variables for binning purpose
size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
size_bins = [0, 1000, 2000, 5000]
scores_by_school_size['School Size'] = pd.cut(scores_by_school_size['Total Students'], bins = size_bins, labels = size_labels, include_lowest = True)

del scores_by_school_size['Total Students']
scores_by_school_size = scores_by_school_size.groupby('School Size').mean()

scores_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Size

In [31]:
# Scores by School Size
scores_by_school_type = {'School Type': school_type, 'Average Math Score': ave_math_score, 'Average Reading Score': ave_reading_score,
                  '% Passing Math': percent_Passing_Math, '% Passing Reading': percent_Passing_Reading, '% Overall Passing': overall_Passing}
scores_by_school_type = pd.DataFrame(scores_by_school_type).reset_index(drop = True)

#  Use groupby instead of binning as bin values will not be numeric
scores_by_school_type = scores_by_school_type.groupby('School Type').mean()

scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


##### TWO OBSERVRABLE TRENDS
----------------------

1. Charter schools have relatively smaller student sizes than their District school counterparts, which has positive effects on their performances in both Math and Reading.
2. Higher spending per student does not necessarily have a positive impact on student performance as is the case in district schools which is negatively impacted by the larger size of the school by student count. Better performing students in Math and Reading have lower spending per head with students spending increasing towards least performing students.